<a href="https://colab.research.google.com/github/bk62/deep-learning-notebooks/blob/main/karpathy/01_Makemore_names_MLP_BatchNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

# Makemore 2 - MLP

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2023-11-16 20:21:51--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.01s   

2023-11-16 20:21:51 (17.3 MB/s) - ‘names.txt’ saved [228145/228145]



In [ ]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8], len(words)

(['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia'],
 32033)

In [ ]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:

  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype


(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [ ]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
from torch import nn

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
class MakeMoreMLP(nn.Module):
  def __init__(self, num_embeddings=len(chars) + 1, embedding_dim=10,
               num_chars=3, hidden_dim=100):
    super().__init__()
    self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
    self.stack = nn.Sequential(
        nn.Linear(embedding_dim * num_chars, hidden_dim),
        nn.Tanh(),
        nn.Linear(hidden_dim, num_embeddings)
    )

  def forward(self, x):
    x = self.embeddings(x).view(-1, 30)
    logits = self.stack(x)
    return logits

model = MakeMoreMLP().to(device)
print(model)

MakeMoreMLP(
  (embeddings): Embedding(27, 10)
  (stack): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=27, bias=True)
  )
)


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class NamesDataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, i):
    return self.X[i], self.Y[i]


batch_size = 64


# Create data loaders.
train_dataloader = DataLoader(NamesDataset(Xtr, Ytr), batch_size=batch_size, drop_last=True)
test_dataloader = DataLoader(NamesDataset(Xte, Yte), batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 3])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.402089  [   64/182625]
loss: 1.879535  [64064/182625]
loss: 2.265327  [128064/182625]
Test Error: 
 Accuracy: 33.0%, Avg loss: 2.137007 

Epoch 2
-------------------------------
loss: 2.383752  [   64/182625]
loss: 1.870460  [64064/182625]
loss: 2.260358  [128064/182625]
Test Error: 
 Accuracy: 33.1%, Avg loss: 2.135966 

Epoch 3
-------------------------------
loss: 2.383617  [   64/182625]
loss: 1.868253  [64064/182625]
loss: 2.258203  [128064/182625]
Test Error: 
 Accuracy: 33.1%, Avg loss: 2.135526 

Epoch 4
-------------------------------
loss: 2.382157  [   64/182625]
loss: 1.867142  [64064/182625]
loss: 2.256966  [128064/182625]
Test Error: 
 Accuracy: 33.1%, Avg loss: 2.135235 

Epoch 5
-------------------------------
loss: 2.380704  [   64/182625]
loss: 1.866384  [64064/182625]
loss: 2.255970  [128064/182625]
Test Error: 
 Accuracy: 33.2%, Avg loss: 2.135003 

Done!


In [ ]:
# sample from the model
g = torch.Generator(device=device).manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      logits = model(torch.tensor([context]).to(device))
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

brenten.
ezrinde.
randrynn.
rer.
ley.
jeidia.
murah.
emi.
chritt.
den.
wylissa.
mely.
breyloni.
abstet.
tylendelidyn.
zaz.
rann.
jullana.
dorge.
driel.


# Makemore 3 - Batch normalization

In [ ]:
class MakeMoreMLPBN(nn.Module):
  def __init__(self, num_embeddings=len(chars) + 1, embedding_dim=10,
               num_chars=3, hidden_dim=100):
    super().__init__()
    self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
    self.stack = nn.Sequential(
        nn.Linear(embedding_dim * num_chars, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, num_embeddings),
        nn.BatchNorm1d(num_embeddings, momentum=0.01)
    )

  def forward(self, x):
    x = self.embeddings(x).view(-1, 30)
    logits = self.stack(x)
    return logits

model = MakeMoreMLPBN().to(device)
print(model)

MakeMoreMLPBN(
  (embeddings): Embedding(27, 10)
  (stack): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=100, out_features=27, bias=True)
    (4): BatchNorm1d(27, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.333525  [   64/182625]
loss: 1.874703  [64064/182625]
loss: 2.207645  [128064/182625]
Test Error: 
 Accuracy: 33.5%, Avg loss: 2.155175 

Epoch 2
-------------------------------
loss: 2.332535  [   64/182625]
loss: 1.874626  [64064/182625]
loss: 2.207402  [128064/182625]
Test Error: 
 Accuracy: 33.5%, Avg loss: 2.155139 

Epoch 3
-------------------------------
loss: 2.332241  [   64/182625]
loss: 1.874454  [64064/182625]
loss: 2.207337  [128064/182625]
Test Error: 
 Accuracy: 33.5%, Avg loss: 2.155130 

Epoch 4
-------------------------------
loss: 2.332170  [   64/182625]
loss: 1.874248  [64064/182625]
loss: 2.207365  [128064/182625]
Test Error: 
 Accuracy: 33.5%, Avg loss: 2.155133 

Epoch 5
-------------------------------
loss: 2.332170  [   64/182625]
loss: 1.874047  [64064/182625]
loss: 2.207435  [128064/182625]
Test Error: 
 Accuracy: 33.5%, Avg loss: 2.155142 

Done!


In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      logits = model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

In [ ]:
class MakeMoreMLPBN(nn.Module):
  def __init__(self, num_embeddings=len(chars) + 1, embedding_dim=10,
               num_chars=3, hidden_dim=100):
    super().__init__()
    self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
    self.stack = nn.Sequential(
        nn.Linear(embedding_dim * num_chars, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.BatchNorm1d(hidden_dim, momentum=0.01),
        nn.Tanh(),
        nn.Linear(hidden_dim, num_embeddings),
        nn.BatchNorm1d(num_embeddings, momentum=0.01)
    )

  def forward(self, x):
    x = self.embeddings(x).view(-1, 30)
    logits = self.stack(x)
    return logits

model = MakeMoreMLPBN().to(device)
print(model)

MakeMoreMLPBN(
  (embeddings): Embedding(27, 10)
  (stack): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (8): Tanh()
    (9): Linear(in_features=100, out_features=100, bias=True)
    (10): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (11): Tanh()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): BatchNorm1d(100, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (14): Tanh()
    (15): Linear(in_features=100, out_features=27, bias=True)
    (16): BatchNo

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.937626  [   64/182625]
loss: 1.628294  [64064/182625]
loss: 1.922525  [128064/182625]
Test Error: 
 Accuracy: 34.3%, Avg loss: 2.113536 

Epoch 2
-------------------------------
loss: 1.933589  [   64/182625]
loss: 1.629084  [64064/182625]
loss: 1.925005  [128064/182625]
Test Error: 
 Accuracy: 34.4%, Avg loss: 2.113359 

Epoch 3
-------------------------------
loss: 1.931669  [   64/182625]
loss: 1.630161  [64064/182625]
loss: 1.925921  [128064/182625]
Test Error: 
 Accuracy: 34.4%, Avg loss: 2.113462 

Epoch 4
-------------------------------
loss: 1.929956  [   64/182625]
loss: 1.630580  [64064/182625]
loss: 1.926156  [128064/182625]
Test Error: 
 Accuracy: 34.3%, Avg loss: 2.113620 

Epoch 5
-------------------------------
loss: 1.928396  [   64/182625]
loss: 1.630594  [64064/182625]
loss: 1.926077  [128064/182625]
Test Error: 
 Accuracy: 34.3%, Avg loss: 2.113789 

Done!


In [ ]:
# sample from the model
g = torch.Generator(device=device).manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      logits = model(torch.tensor([context]).to(device))
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

brent.
shauniah.
frandro.
hardrick.
breidyn.
murphyne.
valyanna.
georgencis.
melo.
jaelynn.
aarsh.
taly.
keng.
chana.
jah.
aan.
jullani.
donna.
aviel.
thi.
